In [1]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/rickwierenga/datasets-1", .branch("iris"))' Datasets
%install '.package(url: "https://github.com/s5tf-team/S5TF", .branch("master"))' S5TF

Installing packages:
	.package(url: "https://github.com/rickwierenga/datasets-1", .branch("iris"))
		Datasets
	.package(url: "https://github.com/s5tf-team/S5TF", .branch("master"))
		S5TF
With SwiftPM flags: []
Working in: /tmp/tmp05snxfm0/swift-install
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[2/3] Merging module jupyterInstalledPackages
Initializing Swift...
Installation complete!


In [0]:
import Datasets
import Foundation
import TensorFlow
import S5TF

In [0]:
let hiddenSize: Int = 10
struct IrisModel: Layer {
    var layer1 = Dense<Float>(inputSize: 4, outputSize: hiddenSize, activation: relu)
    var layer2 = Dense<Float>(inputSize: hiddenSize, outputSize: hiddenSize, activation: relu)
    var layer3 = Dense<Float>(inputSize: hiddenSize, outputSize: 3)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: layer1, layer2, layer3)
    }
}

var model = IrisModel()

In [0]:
let optimizer = SGD(for: model, learningRate: 0.01)

In [0]:
var trainAccuracyResults: [Float] = []
var trainLossResults: [Float] = []

In [6]:
func accuracy(predictions: Tensor<Int32>, truths: Tensor<Int32>) -> Float {
    return Tensor<Float>(predictions .== truths).mean().scalarized()
}

for epoch in 1...25 {
    var epochLoss: Float = 0
    var epochAccuracy: Float = 0
    var batchCount: Int = 0
    for batch in Iris.train.batched(32) {
        let (loss, grad) = model.valueWithGradient { (model: IrisModel) -> Tensor<Float> in
            let logits = model(batch.data)
            return softmaxCrossEntropy(logits: logits, labels: batch.labels)
        }
        optimizer.update(&model, along: grad)
        
        let logits = model(batch.data)
        epochAccuracy += accuracy(predictions: logits.argmax(squeezingAxis: 1), truths: batch.labels)
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochAccuracy /= Float(batchCount)
    epochLoss /= Float(batchCount)
    trainAccuracyResults.append(epochAccuracy)
    trainLossResults.append(epochLoss)
    print("Epoch \(epoch): Loss: \(epochLoss), Accuracy: \(epochAccuracy)")
}

Epoch 1: Loss: 1.3914363, Accuracy: 0.0234375
Epoch 2: Loss: 1.2372257, Accuracy: 0.36292613
Epoch 3: Loss: 1.1511984, Accuracy: 0.49289772
Epoch 4: Loss: 1.0948713, Accuracy: 0.59517044
Epoch 5: Loss: 1.0546119, Accuracy: 0.62642044
Epoch 6: Loss: 1.0223342, Accuracy: 0.64204544
Epoch 7: Loss: 0.9949523, Accuracy: 0.6924716
Epoch 8: Loss: 0.9706927, Accuracy: 0.74289775
Epoch 9: Loss: 0.94875807, Accuracy: 0.78196025
Epoch 10: Loss: 0.9278655, Accuracy: 0.8011364
Epoch 11: Loss: 0.90821826, Accuracy: 0.8558239
Epoch 12: Loss: 0.88903487, Accuracy: 0.9026989
Epoch 13: Loss: 0.87055373, Accuracy: 0.953125
Epoch 14: Loss: 0.85318387, Accuracy: 0.9609375
Epoch 15: Loss: 0.83586925, Accuracy: 0.96875
Epoch 16: Loss: 0.81875503, Accuracy: 0.984375
Epoch 17: Loss: 0.80204326, Accuracy: 0.9921875
Epoch 18: Loss: 0.7857868, Accuracy: 0.9921875
Epoch 19: Loss: 0.7704154, Accuracy: 0.9921875
Epoch 20: Loss: 0.7545458, Accuracy: 0.9921875
Epoch 21: Loss: 0.73898983, Accuracy: 0.9921875
Epoch 22: 